In [2]:
import sympy as sym
import matplotlib.pyplot as plt
import os

In [8]:
S_c, S_d, I_c, I_d = sym.symbols('S_c S_d I_c I_d')
beta, alpha, gamma = sym.symbols('beta alpha gamma')
sigma_d, sigma_c = sym.symbols('sigma_d sigma_c')
r_cc, r_cd, r_dc, r_dd = sym.symbols('r_cc, r_cd, r_dc, r_dd')
s_cc, s_cd, s_dc, s_dd = sym.symbols('s_cc, s_cd, s_dc, s_dd')
t_cc, t_cd, t_dc, t_dd = sym.symbols('t_cc, t_cd, t_dc, t_dd')
p_cc, p_cd, p_dc, p_dd = sym.symbols('p_cc, p_cd, p_dc, p_dd')

f_sc = r_cc*S_c + r_cd*S_d + (s_cc - sigma_c*(S_c + S_d))*I_c + (s_cd - sigma_c*(S_c + S_d))*I_d
f_sd = r_dc*S_c + r_dd*S_d + (s_dc - sigma_d*(I_c + I_d))*I_c + (s_dd - sigma_d*(I_c + I_d))*I_d
f_ic = (t_cc - sigma_c*(S_c + S_d))*S_c + (t_cd - sigma_c*(S_c+S_d))*S_d + p_cc*I_c + p_cd*I_d
f_id = (t_dc - sigma_d*(I_c + I_d))*S_c + (t_dd - sigma_d*(I_c+I_d))*S_d + p_dc*I_c + p_dd*I_d

fbar_s = f_sc*S_c/(S_c + S_d) + f_sd*S_d/(S_c + S_d) 
fbar_i = f_ic*I_c/(I_c + I_d) + f_id*I_d/(I_c + I_d)

F = sym.Matrix([beta*S_c*(I_d + alpha*I_c), beta*S_d*(I_d + alpha*I_c)])
V = sym.Matrix([gamma*I_c - I_c*(f_ic - fbar_i), gamma*I_d - I_d*(f_id - fbar_i)])

print(f'Flujos de entrada a la enfermedad')
sym.print_latex(F)
print('\n')
print(f'Flujos dentro de la enfermedad y de salidad')
sym.print_latex(V)


Flujos de entrada a la enfermedad
\left[\begin{matrix}S_{c} \beta \left(I_{c} \alpha + I_{d}\right)\\S_{d} \beta \left(I_{c} \alpha + I_{d}\right)\end{matrix}\right]


Flujos dentro de la enfermedad y de salidad
\left[\begin{matrix}I_{c} \gamma - I_{c} \left(I_{c} p_{cc} - \frac{I_{c} \left(I_{c} p_{cc} + I_{d} p_{cd} + S_{c} \left(- \sigma_{c} \left(S_{c} + S_{d}\right) + t_{cc}\right) + S_{d} \left(- \sigma_{c} \left(S_{c} + S_{d}\right) + t_{cd}\right)\right)}{I_{c} + I_{d}} + I_{d} p_{cd} - \frac{I_{d} \left(I_{c} p_{dc} + I_{d} p_{dd} + S_{c} \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dc}\right) + S_{d} \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dd}\right)\right)}{I_{c} + I_{d}} + S_{c} \left(- \sigma_{c} \left(S_{c} + S_{d}\right) + t_{cc}\right) + S_{d} \left(- \sigma_{c} \left(S_{c} + S_{d}\right) + t_{cd}\right)\right)\\I_{d} \gamma - I_{d} \left(I_{c} p_{dc} - \frac{I_{c} \left(I_{c} p_{cc} + I_{d} p_{cd} + S_{c} \left(- \sigma_{c} \left(S_{c} + S_{d}\right) +

$$ F = \left[\begin{matrix}S_{c} \beta \left(I_{c} \alpha + I_{d}\right)\\S_{d} \beta \left(I_{c} \alpha + I_{d}\right)\end{matrix}\right]$$ 

$$ V = \left[\begin{matrix}I_{c} \gamma - I_{c} \left(I_{c} p_{cc} - \frac{I_{c} \left(I_{c} p_{cc} + I_{d} p_{cd} + S_{c} \left(- \sigma_{c} \left(S_{c} + S_{d}\right) + t_{cc}\right) + S_{d} \left(- \sigma_{c} \left(S_{c} + S_{d}\right) + t_{cd}\right)\right)}{I_{c} + I_{d}} + I_{d} p_{cd} - \frac{I_{d} \left(I_{c} p_{dc} + I_{d} p_{dd} + S_{c} \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dc}\right) + S_{d} \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dd}\right)\right)}{I_{c} + I_{d}} + S_{c} \left(- \sigma_{c} \left(S_{c} + S_{d}\right) + t_{cc}\right) + S_{d} \left(- \sigma_{c} \left(S_{c} + S_{d}\right) + t_{cd}\right)\right)\\I_{d} \gamma - I_{d} \left(I_{c} p_{dc} - \frac{I_{c} \left(I_{c} p_{cc} + I_{d} p_{cd} + S_{c} \left(- \sigma_{c} \left(S_{c} + S_{d}\right) + t_{cc}\right) + S_{d} \left(- \sigma_{c} \left(S_{c} + S_{d}\right) + t_{cd}\right)\right)}{I_{c} + I_{d}} + I_{d} p_{dd} - \frac{I_{d} \left(I_{c} p_{dc} + I_{d} p_{dd} + S_{c} \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dc}\right) + S_{d} \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dd}\right)\right)}{I_{c} + I_{d}} + S_{c} \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dc}\right) + S_{d} \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dd}\right)\right)\end{matrix}\right] $$


In [10]:
J_F = F.jacobian([I_c, I_d])
J_V = V.jacobian([I_c, I_d])

print(f'Jacobiano de los Flujos de entrada a la enfermedad')
sym.print_latex(sym.simplify(J_F))
print('\n')
print(f'Jacobiano de los Flujos dentro de la enfermedad y de salidad')
sym.print_latex(sym.simplify(J_V))

Jacobiano de los Flujos de entrada a la enfermedad
\left[\begin{matrix}S_{c} \alpha \beta & S_{c} \beta\\S_{d} \alpha \beta & S_{d} \beta\end{matrix}\right]


Jacobiano de los Flujos dentro de la enfermedad y de salidad
\left[\begin{matrix}\frac{- I_{c}^{2} I_{d} S_{c} \sigma_{d} - I_{c}^{2} I_{d} S_{d} \sigma_{d} - I_{c}^{2} I_{d} p_{cc} + I_{c}^{2} I_{d} p_{dc} + I_{c}^{2} \gamma - 2 I_{c} I_{d}^{2} S_{c} \sigma_{d} - 2 I_{c} I_{d}^{2} S_{d} \sigma_{d} - 2 I_{c} I_{d}^{2} p_{cc} + 2 I_{c} I_{d}^{2} p_{dc} + 2 I_{c} I_{d} \gamma - I_{d}^{3} S_{c} \sigma_{d} - I_{d}^{3} S_{d} \sigma_{d} - I_{d}^{3} p_{cd} + I_{d}^{3} p_{dd} + I_{d}^{2} S_{c}^{2} \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \sigma_{c} - I_{d}^{2} S_{c} t_{cc} + I_{d}^{2} S_{c} t_{dc} + I_{d}^{2} S_{d}^{2} \sigma_{c} - I_{d}^{2} S_{d} t_{cd} + I_{d}^{2} S_{d} t_{dd} + I_{d}^{2} \gamma}{I_{c}^{2} + 2 I_{c} I_{d} + I_{d}^{2}} & \frac{I_{c} \left(- I_{c} \left(I_{c} p_{cc} + I_{d} p_{cd} - S_{c} \left(\sigma_{c} \left(S_{c} + S_{d}

$$ J_{F} = \left[\begin{matrix}S_{c} \alpha \beta & S_{c} \beta\\S_{d} \alpha \beta & S_{d} \beta\end{matrix}\right] $$

$$ J_{V} = \left[\begin{matrix}\frac{- I_{c}^{2} I_{d} S_{c} \sigma_{d} - I_{c}^{2} I_{d} S_{d} \sigma_{d} - I_{c}^{2} I_{d} p_{cc} + I_{c}^{2} I_{d} p_{dc} + I_{c}^{2} \gamma - 2 I_{c} I_{d}^{2} S_{c} \sigma_{d} - 2 I_{c} I_{d}^{2} S_{d} \sigma_{d} - 2 I_{c} I_{d}^{2} p_{cc} + 2 I_{c} I_{d}^{2} p_{dc} + 2 I_{c} I_{d} \gamma - I_{d}^{3} S_{c} \sigma_{d} - I_{d}^{3} S_{d} \sigma_{d} - I_{d}^{3} p_{cd} + I_{d}^{3} p_{dd} + I_{d}^{2} S_{c}^{2} \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \sigma_{c} - I_{d}^{2} S_{c} t_{cc} + I_{d}^{2} S_{c} t_{dc} + I_{d}^{2} S_{d}^{2} \sigma_{c} - I_{d}^{2} S_{d} t_{cd} + I_{d}^{2} S_{d} t_{dd} + I_{d}^{2} \gamma}{I_{c}^{2} + 2 I_{c} I_{d} + I_{d}^{2}} & \frac{I_{c} \left(- I_{c} \left(I_{c} p_{cc} + I_{d} p_{cd} - S_{c} \left(\sigma_{c} \left(S_{c} + S_{d}\right) - t_{cc}\right) - S_{d} \left(\sigma_{c} \left(S_{c} + S_{d}\right) - t_{cd}\right)\right) - I_{d} \left(I_{c} p_{dc} + I_{d} p_{dd} - S_{c} \left(\sigma_{d} \left(I_{c} + I_{d}\right) - t_{dc}\right) - S_{d} \left(\sigma_{d} \left(I_{c} + I_{d}\right) - t_{dd}\right)\right) - p_{cd} \left(I_{c} + I_{d}\right)^{2} + \left(I_{c} + I_{d}\right) \left(I_{c} p_{cd} + I_{c} p_{dc} + I_{d} p_{dd} - I_{d} \left(S_{c} \sigma_{d} + S_{d} \sigma_{d} - p_{dd}\right) - S_{c} \left(\sigma_{d} \left(I_{c} + I_{d}\right) - t_{dc}\right) - S_{d} \left(\sigma_{d} \left(I_{c} + I_{d}\right) - t_{dd}\right)\right)\right)}{\left(I_{c} + I_{d}\right)^{2}}\\\frac{I_{d} \left(- I_{c} \left(I_{c} p_{cc} + I_{d} p_{cd} - S_{c} \left(\sigma_{c} \left(S_{c} + S_{d}\right) - t_{cc}\right) - S_{d} \left(\sigma_{c} \left(S_{c} + S_{d}\right) - t_{cd}\right)\right) - I_{d} \left(I_{c} p_{dc} + I_{d} p_{dd} - S_{c} \left(\sigma_{d} \left(I_{c} + I_{d}\right) - t_{dc}\right) - S_{d} \left(\sigma_{d} \left(I_{c} + I_{d}\right) - t_{dd}\right)\right) + \left(I_{c} + I_{d}\right)^{2} \left(S_{c} \sigma_{d} + S_{d} \sigma_{d} - p_{dc}\right) + \left(I_{c} + I_{d}\right) \left(2 I_{c} p_{cc} + I_{d} p_{cd} - I_{d} \left(S_{c} \sigma_{d} + S_{d} \sigma_{d} - p_{dc}\right) - S_{c} \left(\sigma_{c} \left(S_{c} + S_{d}\right) - t_{cc}\right) - S_{d} \left(\sigma_{c} \left(S_{c} + S_{d}\right) - t_{cd}\right)\right)\right)}{\left(I_{c} + I_{d}\right)^{2}} & \frac{I_{c}^{3} S_{c} \sigma_{d} + I_{c}^{3} S_{d} \sigma_{d} + I_{c}^{3} p_{cc} - I_{c}^{3} p_{dc} + 2 I_{c}^{2} I_{d} S_{c} \sigma_{d} + 2 I_{c}^{2} I_{d} S_{d} \sigma_{d} + 2 I_{c}^{2} I_{d} p_{cd} - 2 I_{c}^{2} I_{d} p_{dd} - I_{c}^{2} S_{c}^{2} \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \sigma_{c} + I_{c}^{2} S_{c} t_{cc} - I_{c}^{2} S_{c} t_{dc} - I_{c}^{2} S_{d}^{2} \sigma_{c} + I_{c}^{2} S_{d} t_{cd} - I_{c}^{2} S_{d} t_{dd} + I_{c}^{2} \gamma + I_{c} I_{d}^{2} S_{c} \sigma_{d} + I_{c} I_{d}^{2} S_{d} \sigma_{d} + I_{c} I_{d}^{2} p_{cd} - I_{c} I_{d}^{2} p_{dd} + 2 I_{c} I_{d} \gamma + I_{d}^{2} \gamma}{I_{c}^{2} + 2 I_{c} I_{d} + I_{d}^{2}}\end{matrix}\right] $$




In [11]:
NGM = J_F*J_V**-1
sym.print_latex(NGM)

\left[\begin{matrix}\frac{S_{c} \alpha \beta \left(I_{c}^{3} S_{c} \sigma_{d} + I_{c}^{3} S_{d} \sigma_{d} + I_{c}^{3} p_{cc} - I_{c}^{3} p_{dc} + 2 I_{c}^{2} I_{d} S_{c} \sigma_{d} + 2 I_{c}^{2} I_{d} S_{d} \sigma_{d} + 2 I_{c}^{2} I_{d} p_{cd} - 2 I_{c}^{2} I_{d} p_{dd} - I_{c}^{2} S_{c}^{2} \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \sigma_{c} + I_{c}^{2} S_{c} t_{cc} - I_{c}^{2} S_{c} t_{dc} - I_{c}^{2} S_{d}^{2} \sigma_{c} + I_{c}^{2} S_{d} t_{cd} - I_{c}^{2} S_{d} t_{dd} + I_{c}^{2} \gamma + I_{c} I_{d}^{2} S_{c} \sigma_{d} + I_{c} I_{d}^{2} S_{d} \sigma_{d} + I_{c} I_{d}^{2} p_{cd} - I_{c} I_{d}^{2} p_{dd} + 2 I_{c} I_{d} \gamma + I_{d}^{2} \gamma\right)}{I_{c}^{3} S_{c} \gamma \sigma_{d} + I_{c}^{3} S_{d} \gamma \sigma_{d} + I_{c}^{3} \gamma p_{cc} - I_{c}^{3} \gamma p_{dc} + I_{c}^{2} I_{d} S_{c} \gamma \sigma_{d} + I_{c}^{2} I_{d} S_{d} \gamma \sigma_{d} - I_{c}^{2} I_{d} \gamma p_{cc} + 2 I_{c}^{2} I_{d} \gamma p_{cd} + I_{c}^{2} I_{d} \gamma p_{dc} - 2 I_{c}^{2} I_{d} \gamma p_{d

$$ NGM = \left[\begin{matrix}\frac{S_{c} \alpha \beta \left(I_{c}^{3} S_{c} \sigma_{d} + I_{c}^{3} S_{d} \sigma_{d} + I_{c}^{3} p_{cc} - I_{c}^{3} p_{dc} + 2 I_{c}^{2} I_{d} S_{c} \sigma_{d} + 2 I_{c}^{2} I_{d} S_{d} \sigma_{d} + 2 I_{c}^{2} I_{d} p_{cd} - 2 I_{c}^{2} I_{d} p_{dd} - I_{c}^{2} S_{c}^{2} \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \sigma_{c} + I_{c}^{2} S_{c} t_{cc} - I_{c}^{2} S_{c} t_{dc} - I_{c}^{2} S_{d}^{2} \sigma_{c} + I_{c}^{2} S_{d} t_{cd} - I_{c}^{2} S_{d} t_{dd} + I_{c}^{2} \gamma + I_{c} I_{d}^{2} S_{c} \sigma_{d} + I_{c} I_{d}^{2} S_{d} \sigma_{d} + I_{c} I_{d}^{2} p_{cd} - I_{c} I_{d}^{2} p_{dd} + 2 I_{c} I_{d} \gamma + I_{d}^{2} \gamma\right)}{I_{c}^{3} S_{c} \gamma \sigma_{d} + I_{c}^{3} S_{d} \gamma \sigma_{d} + I_{c}^{3} \gamma p_{cc} - I_{c}^{3} \gamma p_{dc} + I_{c}^{2} I_{d} S_{c} \gamma \sigma_{d} + I_{c}^{2} I_{d} S_{d} \gamma \sigma_{d} - I_{c}^{2} I_{d} \gamma p_{cc} + 2 I_{c}^{2} I_{d} \gamma p_{cd} + I_{c}^{2} I_{d} \gamma p_{dc} - 2 I_{c}^{2} I_{d} \gamma p_{dd} - I_{c}^{2} S_{c}^{2} \gamma \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \gamma \sigma_{c} + I_{c}^{2} S_{c} \gamma t_{cc} - I_{c}^{2} S_{c} \gamma t_{dc} - I_{c}^{2} S_{d}^{2} \gamma \sigma_{c} + I_{c}^{2} S_{d} \gamma t_{cd} - I_{c}^{2} S_{d} \gamma t_{dd} + I_{c}^{2} \gamma^{2} - I_{c} I_{d}^{2} S_{c} \gamma \sigma_{d} - I_{c} I_{d}^{2} S_{d} \gamma \sigma_{d} - 2 I_{c} I_{d}^{2} \gamma p_{cc} + I_{c} I_{d}^{2} \gamma p_{cd} + 2 I_{c} I_{d}^{2} \gamma p_{dc} - I_{c} I_{d}^{2} \gamma p_{dd} + 2 I_{c} I_{d} \gamma^{2} - I_{d}^{3} S_{c} \gamma \sigma_{d} - I_{d}^{3} S_{d} \gamma \sigma_{d} - I_{d}^{3} \gamma p_{cd} + I_{d}^{3} \gamma p_{dd} + I_{d}^{2} S_{c}^{2} \gamma \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \gamma \sigma_{c} - I_{d}^{2} S_{c} \gamma t_{cc} + I_{d}^{2} S_{c} \gamma t_{dc} + I_{d}^{2} S_{d}^{2} \gamma \sigma_{c} - I_{d}^{2} S_{d} \gamma t_{cd} + I_{d}^{2} S_{d} \gamma t_{dd} + I_{d}^{2} \gamma^{2}} + \frac{S_{c} \beta \left(- I_{c}^{2} I_{d} S_{c} \sigma_{d} - I_{c}^{2} I_{d} S_{d} \sigma_{d} - I_{c}^{2} I_{d} p_{cc} + I_{c}^{2} I_{d} p_{dc} - 2 I_{c} I_{d}^{2} S_{c} \sigma_{d} - 2 I_{c} I_{d}^{2} S_{d} \sigma_{d} - 2 I_{c} I_{d}^{2} p_{cc} + 2 I_{c} I_{d}^{2} p_{dc} - I_{d}^{3} S_{c} \sigma_{d} - I_{d}^{3} S_{d} \sigma_{d} - I_{d}^{3} p_{cd} + I_{d}^{3} p_{dd} + I_{d}^{2} S_{c}^{2} \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \sigma_{c} - I_{d}^{2} S_{c} t_{cc} + I_{d}^{2} S_{c} t_{dc} + I_{d}^{2} S_{d}^{2} \sigma_{c} - I_{d}^{2} S_{d} t_{cd} + I_{d}^{2} S_{d} t_{dd}\right)}{I_{c}^{3} S_{c} \gamma \sigma_{d} + I_{c}^{3} S_{d} \gamma \sigma_{d} + I_{c}^{3} \gamma p_{cc} - I_{c}^{3} \gamma p_{dc} + I_{c}^{2} I_{d} S_{c} \gamma \sigma_{d} + I_{c}^{2} I_{d} S_{d} \gamma \sigma_{d} - I_{c}^{2} I_{d} \gamma p_{cc} + 2 I_{c}^{2} I_{d} \gamma p_{cd} + I_{c}^{2} I_{d} \gamma p_{dc} - 2 I_{c}^{2} I_{d} \gamma p_{dd} - I_{c}^{2} S_{c}^{2} \gamma \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \gamma \sigma_{c} + I_{c}^{2} S_{c} \gamma t_{cc} - I_{c}^{2} S_{c} \gamma t_{dc} - I_{c}^{2} S_{d}^{2} \gamma \sigma_{c} + I_{c}^{2} S_{d} \gamma t_{cd} - I_{c}^{2} S_{d} \gamma t_{dd} + I_{c}^{2} \gamma^{2} - I_{c} I_{d}^{2} S_{c} \gamma \sigma_{d} - I_{c} I_{d}^{2} S_{d} \gamma \sigma_{d} - 2 I_{c} I_{d}^{2} \gamma p_{cc} + I_{c} I_{d}^{2} \gamma p_{cd} + 2 I_{c} I_{d}^{2} \gamma p_{dc} - I_{c} I_{d}^{2} \gamma p_{dd} + 2 I_{c} I_{d} \gamma^{2} - I_{d}^{3} S_{c} \gamma \sigma_{d} - I_{d}^{3} S_{d} \gamma \sigma_{d} - I_{d}^{3} \gamma p_{cd} + I_{d}^{3} \gamma p_{dd} + I_{d}^{2} S_{c}^{2} \gamma \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \gamma \sigma_{c} - I_{d}^{2} S_{c} \gamma t_{cc} + I_{d}^{2} S_{c} \gamma t_{dc} + I_{d}^{2} S_{d}^{2} \gamma \sigma_{c} - I_{d}^{2} S_{d} \gamma t_{cd} + I_{d}^{2} S_{d} \gamma t_{dd} + I_{d}^{2} \gamma^{2}} & \frac{S_{c} \alpha \beta \left(I_{c}^{3} S_{c} \sigma_{d} + I_{c}^{3} S_{d} \sigma_{d} + I_{c}^{3} p_{cc} - I_{c}^{3} p_{dc} + 2 I_{c}^{2} I_{d} S_{c} \sigma_{d} + 2 I_{c}^{2} I_{d} S_{d} \sigma_{d} + 2 I_{c}^{2} I_{d} p_{cd} - 2 I_{c}^{2} I_{d} p_{dd} - I_{c}^{2} S_{c}^{2} \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \sigma_{c} + I_{c}^{2} S_{c} t_{cc} - I_{c}^{2} S_{c} t_{dc} - I_{c}^{2} S_{d}^{2} \sigma_{c} + I_{c}^{2} S_{d} t_{cd} - I_{c}^{2} S_{d} t_{dd} + I_{c} I_{d}^{2} S_{c} \sigma_{d} + I_{c} I_{d}^{2} S_{d} \sigma_{d} + I_{c} I_{d}^{2} p_{cd} - I_{c} I_{d}^{2} p_{dd}\right)}{I_{c}^{3} S_{c} \gamma \sigma_{d} + I_{c}^{3} S_{d} \gamma \sigma_{d} + I_{c}^{3} \gamma p_{cc} - I_{c}^{3} \gamma p_{dc} + I_{c}^{2} I_{d} S_{c} \gamma \sigma_{d} + I_{c}^{2} I_{d} S_{d} \gamma \sigma_{d} - I_{c}^{2} I_{d} \gamma p_{cc} + 2 I_{c}^{2} I_{d} \gamma p_{cd} + I_{c}^{2} I_{d} \gamma p_{dc} - 2 I_{c}^{2} I_{d} \gamma p_{dd} - I_{c}^{2} S_{c}^{2} \gamma \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \gamma \sigma_{c} + I_{c}^{2} S_{c} \gamma t_{cc} - I_{c}^{2} S_{c} \gamma t_{dc} - I_{c}^{2} S_{d}^{2} \gamma \sigma_{c} + I_{c}^{2} S_{d} \gamma t_{cd} - I_{c}^{2} S_{d} \gamma t_{dd} + I_{c}^{2} \gamma^{2} - I_{c} I_{d}^{2} S_{c} \gamma \sigma_{d} - I_{c} I_{d}^{2} S_{d} \gamma \sigma_{d} - 2 I_{c} I_{d}^{2} \gamma p_{cc} + I_{c} I_{d}^{2} \gamma p_{cd} + 2 I_{c} I_{d}^{2} \gamma p_{dc} - I_{c} I_{d}^{2} \gamma p_{dd} + 2 I_{c} I_{d} \gamma^{2} - I_{d}^{3} S_{c} \gamma \sigma_{d} - I_{d}^{3} S_{d} \gamma \sigma_{d} - I_{d}^{3} \gamma p_{cd} + I_{d}^{3} \gamma p_{dd} + I_{d}^{2} S_{c}^{2} \gamma \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \gamma \sigma_{c} - I_{d}^{2} S_{c} \gamma t_{cc} + I_{d}^{2} S_{c} \gamma t_{dc} + I_{d}^{2} S_{d}^{2} \gamma \sigma_{c} - I_{d}^{2} S_{d} \gamma t_{cd} + I_{d}^{2} S_{d} \gamma t_{dd} + I_{d}^{2} \gamma^{2}} + \frac{S_{c} \beta \left(- I_{c}^{2} I_{d} S_{c} \sigma_{d} - I_{c}^{2} I_{d} S_{d} \sigma_{d} - I_{c}^{2} I_{d} p_{cc} + I_{c}^{2} I_{d} p_{dc} + I_{c}^{2} \gamma - 2 I_{c} I_{d}^{2} S_{c} \sigma_{d} - 2 I_{c} I_{d}^{2} S_{d} \sigma_{d} - 2 I_{c} I_{d}^{2} p_{cc} + 2 I_{c} I_{d}^{2} p_{dc} + 2 I_{c} I_{d} \gamma - I_{d}^{3} S_{c} \sigma_{d} - I_{d}^{3} S_{d} \sigma_{d} - I_{d}^{3} p_{cd} + I_{d}^{3} p_{dd} + I_{d}^{2} S_{c}^{2} \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \sigma_{c} - I_{d}^{2} S_{c} t_{cc} + I_{d}^{2} S_{c} t_{dc} + I_{d}^{2} S_{d}^{2} \sigma_{c} - I_{d}^{2} S_{d} t_{cd} + I_{d}^{2} S_{d} t_{dd} + I_{d}^{2} \gamma\right)}{I_{c}^{3} S_{c} \gamma \sigma_{d} + I_{c}^{3} S_{d} \gamma \sigma_{d} + I_{c}^{3} \gamma p_{cc} - I_{c}^{3} \gamma p_{dc} + I_{c}^{2} I_{d} S_{c} \gamma \sigma_{d} + I_{c}^{2} I_{d} S_{d} \gamma \sigma_{d} - I_{c}^{2} I_{d} \gamma p_{cc} + 2 I_{c}^{2} I_{d} \gamma p_{cd} + I_{c}^{2} I_{d} \gamma p_{dc} - 2 I_{c}^{2} I_{d} \gamma p_{dd} - I_{c}^{2} S_{c}^{2} \gamma \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \gamma \sigma_{c} + I_{c}^{2} S_{c} \gamma t_{cc} - I_{c}^{2} S_{c} \gamma t_{dc} - I_{c}^{2} S_{d}^{2} \gamma \sigma_{c} + I_{c}^{2} S_{d} \gamma t_{cd} - I_{c}^{2} S_{d} \gamma t_{dd} + I_{c}^{2} \gamma^{2} - I_{c} I_{d}^{2} S_{c} \gamma \sigma_{d} - I_{c} I_{d}^{2} S_{d} \gamma \sigma_{d} - 2 I_{c} I_{d}^{2} \gamma p_{cc} + I_{c} I_{d}^{2} \gamma p_{cd} + 2 I_{c} I_{d}^{2} \gamma p_{dc} - I_{c} I_{d}^{2} \gamma p_{dd} + 2 I_{c} I_{d} \gamma^{2} - I_{d}^{3} S_{c} \gamma \sigma_{d} - I_{d}^{3} S_{d} \gamma \sigma_{d} - I_{d}^{3} \gamma p_{cd} + I_{d}^{3} \gamma p_{dd} + I_{d}^{2} S_{c}^{2} \gamma \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \gamma \sigma_{c} - I_{d}^{2} S_{c} \gamma t_{cc} + I_{d}^{2} S_{c} \gamma t_{dc} + I_{d}^{2} S_{d}^{2} \gamma \sigma_{c} - I_{d}^{2} S_{d} \gamma t_{cd} + I_{d}^{2} S_{d} \gamma t_{dd} + I_{d}^{2} \gamma^{2}}\\\frac{S_{d} \alpha \beta \left(I_{c}^{3} S_{c} \sigma_{d} + I_{c}^{3} S_{d} \sigma_{d} + I_{c}^{3} p_{cc} - I_{c}^{3} p_{dc} + 2 I_{c}^{2} I_{d} S_{c} \sigma_{d} + 2 I_{c}^{2} I_{d} S_{d} \sigma_{d} + 2 I_{c}^{2} I_{d} p_{cd} - 2 I_{c}^{2} I_{d} p_{dd} - I_{c}^{2} S_{c}^{2} \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \sigma_{c} + I_{c}^{2} S_{c} t_{cc} - I_{c}^{2} S_{c} t_{dc} - I_{c}^{2} S_{d}^{2} \sigma_{c} + I_{c}^{2} S_{d} t_{cd} - I_{c}^{2} S_{d} t_{dd} + I_{c}^{2} \gamma + I_{c} I_{d}^{2} S_{c} \sigma_{d} + I_{c} I_{d}^{2} S_{d} \sigma_{d} + I_{c} I_{d}^{2} p_{cd} - I_{c} I_{d}^{2} p_{dd} + 2 I_{c} I_{d} \gamma + I_{d}^{2} \gamma\right)}{I_{c}^{3} S_{c} \gamma \sigma_{d} + I_{c}^{3} S_{d} \gamma \sigma_{d} + I_{c}^{3} \gamma p_{cc} - I_{c}^{3} \gamma p_{dc} + I_{c}^{2} I_{d} S_{c} \gamma \sigma_{d} + I_{c}^{2} I_{d} S_{d} \gamma \sigma_{d} - I_{c}^{2} I_{d} \gamma p_{cc} + 2 I_{c}^{2} I_{d} \gamma p_{cd} + I_{c}^{2} I_{d} \gamma p_{dc} - 2 I_{c}^{2} I_{d} \gamma p_{dd} - I_{c}^{2} S_{c}^{2} \gamma \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \gamma \sigma_{c} + I_{c}^{2} S_{c} \gamma t_{cc} - I_{c}^{2} S_{c} \gamma t_{dc} - I_{c}^{2} S_{d}^{2} \gamma \sigma_{c} + I_{c}^{2} S_{d} \gamma t_{cd} - I_{c}^{2} S_{d} \gamma t_{dd} + I_{c}^{2} \gamma^{2} - I_{c} I_{d}^{2} S_{c} \gamma \sigma_{d} - I_{c} I_{d}^{2} S_{d} \gamma \sigma_{d} - 2 I_{c} I_{d}^{2} \gamma p_{cc} + I_{c} I_{d}^{2} \gamma p_{cd} + 2 I_{c} I_{d}^{2} \gamma p_{dc} - I_{c} I_{d}^{2} \gamma p_{dd} + 2 I_{c} I_{d} \gamma^{2} - I_{d}^{3} S_{c} \gamma \sigma_{d} - I_{d}^{3} S_{d} \gamma \sigma_{d} - I_{d}^{3} \gamma p_{cd} + I_{d}^{3} \gamma p_{dd} + I_{d}^{2} S_{c}^{2} \gamma \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \gamma \sigma_{c} - I_{d}^{2} S_{c} \gamma t_{cc} + I_{d}^{2} S_{c} \gamma t_{dc} + I_{d}^{2} S_{d}^{2} \gamma \sigma_{c} - I_{d}^{2} S_{d} \gamma t_{cd} + I_{d}^{2} S_{d} \gamma t_{dd} + I_{d}^{2} \gamma^{2}} + \frac{S_{d} \beta \left(- I_{c}^{2} I_{d} S_{c} \sigma_{d} - I_{c}^{2} I_{d} S_{d} \sigma_{d} - I_{c}^{2} I_{d} p_{cc} + I_{c}^{2} I_{d} p_{dc} - 2 I_{c} I_{d}^{2} S_{c} \sigma_{d} - 2 I_{c} I_{d}^{2} S_{d} \sigma_{d} - 2 I_{c} I_{d}^{2} p_{cc} + 2 I_{c} I_{d}^{2} p_{dc} - I_{d}^{3} S_{c} \sigma_{d} - I_{d}^{3} S_{d} \sigma_{d} - I_{d}^{3} p_{cd} + I_{d}^{3} p_{dd} + I_{d}^{2} S_{c}^{2} \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \sigma_{c} - I_{d}^{2} S_{c} t_{cc} + I_{d}^{2} S_{c} t_{dc} + I_{d}^{2} S_{d}^{2} \sigma_{c} - I_{d}^{2} S_{d} t_{cd} + I_{d}^{2} S_{d} t_{dd}\right)}{I_{c}^{3} S_{c} \gamma \sigma_{d} + I_{c}^{3} S_{d} \gamma \sigma_{d} + I_{c}^{3} \gamma p_{cc} - I_{c}^{3} \gamma p_{dc} + I_{c}^{2} I_{d} S_{c} \gamma \sigma_{d} + I_{c}^{2} I_{d} S_{d} \gamma \sigma_{d} - I_{c}^{2} I_{d} \gamma p_{cc} + 2 I_{c}^{2} I_{d} \gamma p_{cd} + I_{c}^{2} I_{d} \gamma p_{dc} - 2 I_{c}^{2} I_{d} \gamma p_{dd} - I_{c}^{2} S_{c}^{2} \gamma \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \gamma \sigma_{c} + I_{c}^{2} S_{c} \gamma t_{cc} - I_{c}^{2} S_{c} \gamma t_{dc} - I_{c}^{2} S_{d}^{2} \gamma \sigma_{c} + I_{c}^{2} S_{d} \gamma t_{cd} - I_{c}^{2} S_{d} \gamma t_{dd} + I_{c}^{2} \gamma^{2} - I_{c} I_{d}^{2} S_{c} \gamma \sigma_{d} - I_{c} I_{d}^{2} S_{d} \gamma \sigma_{d} - 2 I_{c} I_{d}^{2} \gamma p_{cc} + I_{c} I_{d}^{2} \gamma p_{cd} + 2 I_{c} I_{d}^{2} \gamma p_{dc} - I_{c} I_{d}^{2} \gamma p_{dd} + 2 I_{c} I_{d} \gamma^{2} - I_{d}^{3} S_{c} \gamma \sigma_{d} - I_{d}^{3} S_{d} \gamma \sigma_{d} - I_{d}^{3} \gamma p_{cd} + I_{d}^{3} \gamma p_{dd} + I_{d}^{2} S_{c}^{2} \gamma \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \gamma \sigma_{c} - I_{d}^{2} S_{c} \gamma t_{cc} + I_{d}^{2} S_{c} \gamma t_{dc} + I_{d}^{2} S_{d}^{2} \gamma \sigma_{c} - I_{d}^{2} S_{d} \gamma t_{cd} + I_{d}^{2} S_{d} \gamma t_{dd} + I_{d}^{2} \gamma^{2}} & \frac{S_{d} \alpha \beta \left(I_{c}^{3} S_{c} \sigma_{d} + I_{c}^{3} S_{d} \sigma_{d} + I_{c}^{3} p_{cc} - I_{c}^{3} p_{dc} + 2 I_{c}^{2} I_{d} S_{c} \sigma_{d} + 2 I_{c}^{2} I_{d} S_{d} \sigma_{d} + 2 I_{c}^{2} I_{d} p_{cd} - 2 I_{c}^{2} I_{d} p_{dd} - I_{c}^{2} S_{c}^{2} \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \sigma_{c} + I_{c}^{2} S_{c} t_{cc} - I_{c}^{2} S_{c} t_{dc} - I_{c}^{2} S_{d}^{2} \sigma_{c} + I_{c}^{2} S_{d} t_{cd} - I_{c}^{2} S_{d} t_{dd} + I_{c} I_{d}^{2} S_{c} \sigma_{d} + I_{c} I_{d}^{2} S_{d} \sigma_{d} + I_{c} I_{d}^{2} p_{cd} - I_{c} I_{d}^{2} p_{dd}\right)}{I_{c}^{3} S_{c} \gamma \sigma_{d} + I_{c}^{3} S_{d} \gamma \sigma_{d} + I_{c}^{3} \gamma p_{cc} - I_{c}^{3} \gamma p_{dc} + I_{c}^{2} I_{d} S_{c} \gamma \sigma_{d} + I_{c}^{2} I_{d} S_{d} \gamma \sigma_{d} - I_{c}^{2} I_{d} \gamma p_{cc} + 2 I_{c}^{2} I_{d} \gamma p_{cd} + I_{c}^{2} I_{d} \gamma p_{dc} - 2 I_{c}^{2} I_{d} \gamma p_{dd} - I_{c}^{2} S_{c}^{2} \gamma \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \gamma \sigma_{c} + I_{c}^{2} S_{c} \gamma t_{cc} - I_{c}^{2} S_{c} \gamma t_{dc} - I_{c}^{2} S_{d}^{2} \gamma \sigma_{c} + I_{c}^{2} S_{d} \gamma t_{cd} - I_{c}^{2} S_{d} \gamma t_{dd} + I_{c}^{2} \gamma^{2} - I_{c} I_{d}^{2} S_{c} \gamma \sigma_{d} - I_{c} I_{d}^{2} S_{d} \gamma \sigma_{d} - 2 I_{c} I_{d}^{2} \gamma p_{cc} + I_{c} I_{d}^{2} \gamma p_{cd} + 2 I_{c} I_{d}^{2} \gamma p_{dc} - I_{c} I_{d}^{2} \gamma p_{dd} + 2 I_{c} I_{d} \gamma^{2} - I_{d}^{3} S_{c} \gamma \sigma_{d} - I_{d}^{3} S_{d} \gamma \sigma_{d} - I_{d}^{3} \gamma p_{cd} + I_{d}^{3} \gamma p_{dd} + I_{d}^{2} S_{c}^{2} \gamma \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \gamma \sigma_{c} - I_{d}^{2} S_{c} \gamma t_{cc} + I_{d}^{2} S_{c} \gamma t_{dc} + I_{d}^{2} S_{d}^{2} \gamma \sigma_{c} - I_{d}^{2} S_{d} \gamma t_{cd} + I_{d}^{2} S_{d} \gamma t_{dd} + I_{d}^{2} \gamma^{2}} + \frac{S_{d} \beta \left(- I_{c}^{2} I_{d} S_{c} \sigma_{d} - I_{c}^{2} I_{d} S_{d} \sigma_{d} - I_{c}^{2} I_{d} p_{cc} + I_{c}^{2} I_{d} p_{dc} + I_{c}^{2} \gamma - 2 I_{c} I_{d}^{2} S_{c} \sigma_{d} - 2 I_{c} I_{d}^{2} S_{d} \sigma_{d} - 2 I_{c} I_{d}^{2} p_{cc} + 2 I_{c} I_{d}^{2} p_{dc} + 2 I_{c} I_{d} \gamma - I_{d}^{3} S_{c} \sigma_{d} - I_{d}^{3} S_{d} \sigma_{d} - I_{d}^{3} p_{cd} + I_{d}^{3} p_{dd} + I_{d}^{2} S_{c}^{2} \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \sigma_{c} - I_{d}^{2} S_{c} t_{cc} + I_{d}^{2} S_{c} t_{dc} + I_{d}^{2} S_{d}^{2} \sigma_{c} - I_{d}^{2} S_{d} t_{cd} + I_{d}^{2} S_{d} t_{dd} + I_{d}^{2} \gamma\right)}{I_{c}^{3} S_{c} \gamma \sigma_{d} + I_{c}^{3} S_{d} \gamma \sigma_{d} + I_{c}^{3} \gamma p_{cc} - I_{c}^{3} \gamma p_{dc} + I_{c}^{2} I_{d} S_{c} \gamma \sigma_{d} + I_{c}^{2} I_{d} S_{d} \gamma \sigma_{d} - I_{c}^{2} I_{d} \gamma p_{cc} + 2 I_{c}^{2} I_{d} \gamma p_{cd} + I_{c}^{2} I_{d} \gamma p_{dc} - 2 I_{c}^{2} I_{d} \gamma p_{dd} - I_{c}^{2} S_{c}^{2} \gamma \sigma_{c} - 2 I_{c}^{2} S_{c} S_{d} \gamma \sigma_{c} + I_{c}^{2} S_{c} \gamma t_{cc} - I_{c}^{2} S_{c} \gamma t_{dc} - I_{c}^{2} S_{d}^{2} \gamma \sigma_{c} + I_{c}^{2} S_{d} \gamma t_{cd} - I_{c}^{2} S_{d} \gamma t_{dd} + I_{c}^{2} \gamma^{2} - I_{c} I_{d}^{2} S_{c} \gamma \sigma_{d} - I_{c} I_{d}^{2} S_{d} \gamma \sigma_{d} - 2 I_{c} I_{d}^{2} \gamma p_{cc} + I_{c} I_{d}^{2} \gamma p_{cd} + 2 I_{c} I_{d}^{2} \gamma p_{dc} - I_{c} I_{d}^{2} \gamma p_{dd} + 2 I_{c} I_{d} \gamma^{2} - I_{d}^{3} S_{c} \gamma \sigma_{d} - I_{d}^{3} S_{d} \gamma \sigma_{d} - I_{d}^{3} \gamma p_{cd} + I_{d}^{3} \gamma p_{dd} + I_{d}^{2} S_{c}^{2} \gamma \sigma_{c} + 2 I_{d}^{2} S_{c} S_{d} \gamma \sigma_{c} - I_{d}^{2} S_{c} \gamma t_{cc} + I_{d}^{2} S_{c} \gamma t_{dc} + I_{d}^{2} S_{d}^{2} \gamma \sigma_{c} - I_{d}^{2} S_{d} \gamma t_{cd} + I_{d}^{2} S_{d} \gamma t_{dd} + I_{d}^{2} \gamma^{2}}\end{matrix}\right] $$


In [13]:
NGM_DFE = NGM.subs([(I_d, 0), (I_c, 0)])
NGM_DFE

Matrix([
[nan, nan],
[nan, nan]])